# Step 1: Reading and Understanding the Data

In [ ]:
#importing libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Step 2: Reading and Checking the Dataset

In [ ]:
# Reading the Dataset
df = pd.read_csv("day.csv")
# Checking the Data
df.head(5)

In [ ]:
# Checking the Description of the Columns of Dataframe
df.describe()

In [ ]:
# Checking the Shape of Dataframe
df.shape

In [ ]:
# Checking the info about columns
df.info()

In [ ]:
# Checking the Null Values
df.isnull().sum()

# Step 3: Mapping

In [ ]:
# Mapping the (season) column.
df['season']=df.season.map({1: 'Spring', 2: 'Summer',3:'Fall', 4:'Winter' })
# Mapping the (month) column.
df['mnth']=df.mnth.map({1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'June',7:'July',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'})
# Mapping the (weathersit) column.
df['weathersit']=df.weathersit.map({1: 'Clear',2:'Mist + Cloudy',3:'Light Snow',4:'Snow + Fog'})
# Mapping the (weekday) column.
df['weekday']=df.weekday.map({0:'Sun',1:'Mon',2:'Tue',3:'Wed',4:'Thu',5:'Fri',6:'Sat'})

In [ ]:
# Dropping unwanted Columns like (instant),(dteday),(atemp),(casual) and (registered) as they are not needed for the analysis or for regression analysis.
df.drop(['instant','dteday','casual','registered','atemp'],axis=1,inplace=True)

In [ ]:
# Checking the Data
df.head(5)

# Step 4: Visualising the Data

In [ ]:
# Visualizing the Categorical Variables of the Dataset using (boxplot): to see how predictor variables stands against the (cnt).
plt.figure(figsize=(20, 12))
plt.subplot(2,4,1)
sns.boxplot(x = 'weekday', y = 'cnt', data = df)
plt.subplot(2,4,2)
sns.boxplot(x = 'holiday', y = 'cnt', data = df)
plt.subplot(2,4,3)
sns.boxplot(x = 'workingday', y = 'cnt', data = df)
plt.subplot(2,4,4)
sns.boxplot(x = 'month', y = 'cnt', data = df)
plt.subplot(2,4,5)
sns.boxplot(x = 'year', y = 'cnt', data = df)
plt.subplot(2,4,6)
sns.boxplot(x = 'season', y = 'cnt', data = df)
plt.subplot(2,4,7)
sns.boxplot(x = 'weathersit', y = 'cnt', data = df)
plt.title('Categorical Variables VS (cnt))
plt.show()

In [ ]:
# Visualizing the Numerical Columns.
sns.pairplot(data=df,vars=['temp','atemp','humidity','windspeed','cnt'])
plt.title('The Numerical Columns')
plt.show()

In [ ]:
# Making a Heatmap to show the correlation between the variables to see if we can perform linear regression on the dataset or not.
plt.figure(figsize=(20, 12))
sns.heatmap(df.corr(), cmap='YlRd', annot=True)
plt.title('The Correlation Between Variables in the Dataset')
plt.show()

We see there are variables correlated to (cnt) such as temp, yr, workingday, so we can thereby conduct a linear regresssion model.

# Step 5: Data Preparation

In [ ]:
# Creating Dummy Variables for (month), (weekday), (weathersit) and (seasons).
month = pd.get_dummies(df.mnth, drop_first=True)
weekday = pd.get_dummies(df.weekday, drop_first=True)
weathersit = pd.get_dummies(df.weathersit, drop_first=True)
season = pd.get_dummies(df.season, drop_first=True)

In [ ]:
# Adding the Dummy Variables to the Original Dataframe
df = pd.concat([df,month, weekday, weathersit, season], axis=1)
df.head(5)

In [ ]:
# Dropping (month), (weekday), (weathersit) and (seasons) as we have created the dummies for it.
df.drop(['season','mnth','weekday','weathersit'], axis = 1, inplace = True)
df.head(5)

In [ ]:
# Checking the Shape of Dataframe after Data Preparation is done.
df.shape

In [ ]:
# Checking the Info about Columns after Data Preparation is done.
df.info()

In [ ]:
# Making a Heatmap to show the Correlation between the new variables in the dataset after Data Preparation is done.
plt.figure(figsize=(20, 12))
sns.heatmap(df.corr(), cmap='YlOrGn', annot=True)
plt.title('Correlation between the Variables in the Dataset after Data Preparation is done')
plt.show()

# Step 6: Preparing the Data for Model Training

In [ ]:
# Splitting the Dataframe into Train and Test sets.
np.random.seed(0)
df_train, df_test = train_test_split(df_new, train_size = 0.7, random_state = 100)

In [ ]:
# Checking the Shape of the Train Dataset
df_train.shape

In [ ]:
# Checking the Shape of the Test Dataset
df_test.shape

In [ ]:
# Rescaling the Variables like (hum), (temp), (windspeed), (cnt) as they have large values Using MinMaxScaler. 
scaler = MinMaxScaler()
scaler_var = ['hum', 'windspeed', 'temp', 'cnt']
df_train[scaler_var] = scaler.fit_transform(df_train[scaler_var])
df_train.head(5)

In [ ]:
# Checking the values of the Train set after performing Scaling.
df_train.describe()

In [ ]:
# Making a Heatmap to show the Correlation coefficients to see which variables are highly correlated after the Data Preparation and Rescaling.
plt.figure(figsize = (25,25))
matrix = np.triu(df_train.corr())
sns.heatmap(df_train.corr(), annot = True, cmap="RdYlGn", mask=matrix)
plt.title('Correlation between the Variables in the Dataset after Data Preparation and Rescaling')
plt.show()

We see (cnt) have correlation with (year) and (temp). Similarly, (Misty) and (humidity) show correlation, Spring with Jan and Feb and Summer with May and Winter with Oct and Nov show good correlation.

In [ ]:
# Visualizing one of the correlation (cnt) and (temp) to see the trends via Scatter plot.
plt.figure(figsize=[6,6])
plt.scatter(df_train.temp, df_train.cnt)
plt.title('Correlation between (cnt) vs (temp)')
plt.show()

Visualization confirms a positive correlation between cnt and temp.

# Step 7: Training the Model

In [ ]:
# Building the Linear Model.
y_train = df_train.pop('cnt')
X_train = df_train

In [ ]:
# Creating the RFE object.
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm, 15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
# List the variables that selected in top 15 list.
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Selecting the selected variable via RFE in col list.
col = X_train.columns[rfe.support_]
print(col)

In [ ]:
# Checking which columns has been rejected.
X_train.columns[~rfe.support_]

In [ ]:
# Make a function to calculate VIF of variables.
def calculateVIF(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif 

In [ ]:
# Dataframe with RFE selected variables and calculate VIF.
X_train_rfe = X_train[col]
calculateVIF(X_train_rfe)

We see tha the (humidity) shows high VIF value.

In [ ]:
# As (humidity) shows high VIF values hence we can drop it
X_train_new = X_train_rfe.drop(['humidity'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

VIF Values seems to be good now, and we will see if we can reduce further.

# Step 8: Building a Linear Models

In [ ]:
# Building the 1st Linear Regression Model.
X_train_lm_1 = sm.add_constant(X_train_new)
lr_1 = sm.OLS(y_train,X_train_lm_1).fit()
print(lr_1.summary())

In [ ]:
# We can drop (Nov) variable as it has high (p-value).
X_train_new = X_train_new.drop(['Nov'], axis = 1)
# Run the function to calculate VIF for the new model.
calculateVIF(X_train_new)

In [ ]:
# Building the 2nd Linear Regression Model.
X_train_lm_2 = sm.add_constant(X_train_new)
lr_2 = sm.OLS(y_train,X_train_lm_2).fit()
print(lr_2.summary())

In [ ]:
# We can drop (Dec) variable as it has high (p-value).
X_train_new = X_train_new.drop(['Dec'], axis = 1)
# Run the function to calculate VIF for the new model.
calculateVIF(X_train_new)

In [ ]:
# Building the 3rd Linear Regression Model.
X_train_lm_3 = sm.add_constant(X_train_new)
lr_3 = sm.OLS(y_train,X_train_lm_3).fit()
print(lr_3.summary())

In [ ]:
# We can drop (Jan) variable as it has high (p-value).
X_train_new = X_train_new.drop(['Jan'], axis = 1)
# Run the function to calculate VIF for the new model.
calculateVIF(X_train_new)

In [ ]:
# Building the 4th Linear Regression Model.
X_train_lm_4 = sm.add_constant(X_train_new)
lr_4 = sm.OLS(y_train,X_train_lm_4).fit()
print(lr_4.summary())

In [ ]:
# We can drop (July) variable as it has high (p-value).
X_train_new = X_train_new.drop(['July'], axis = 1)
# Run the function to calculate VIF for the new model.
calculateVIF(X_train_new)

In [ ]:
# Building the 5th Linear Regression Model.
X_train_lm_5 = sm.add_constant(X_train_new)
lr_5 = sm.OLS(y_train,X_train_lm_5).fit()
print(lr_5.summary())

We can cosider the above model i.e lr_5, as it seems to have very low multicolinearity between the predictors and the p-values for all the predictors seems to be significant.
F-Statistics value of 248.4 (which is greater than 1) and the p-value of 1.47e-186 i.e almost equals to zero, states that the overall model is significant.

In [ ]:
# Checking the Parameters and their Coefficient values
lr_5.params

##### Several points to be noted as we select this model as the final model:

1) The model selection depends on several factor such as the p-value, the VIF and the R-squared value. The p-value gives us input on the significance of the variables, the VIF about the correaltion between the participating variables and the R-squared value gives us an indication about the strength of the model. This value defines the percentage of the variance in the dependent variable that the independent variables explain collectively.

2) The low (<0.05) or almost zero p-value of all the selected variables enables us to reject the null hypothesis. 

3) The VIF should be generally <5 and we have achieved that condition with all the variables. 

4) The R-squared value achieved is 82.7% which suggests a high correlation between the dependent variable (count) and the independent variables and the variables selected accurately help us map the variance of the dependent variable ie count. 

# Step 9: Residual Analysis of the Train data and Validation

In [ ]:
X_train_lm_5

In [ ]:
# Getting the y_train_pred for Residual Analysis.
y_train_pred = lr_5.predict(X_train_lm_5)

#### Normality of Error Terms

In [ ]:
# Histogram of the Error Terms.
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20) 
plt.xlabel('Errors', fontsize = 18)

Error Terms are following a normal distribution.

#### Multi Colinearity

In [ ]:
calculateVIF(X_train_new)

In [ ]:
# Making a Heatmap to show the Correlation X Train.
plt.figure(figsize=(15,8))
sns.heatmap(X_train_new.corr(),annot = True, cmap="RdYlGn")
plt.show()

VIF values are less than 5, so it's good and there is no multicolinearity are seen from the Heatmap.

#### Linearity 

In [ ]:
# Linear Relationship Validation Using CCPR plot.
sm.graphics.plot_ccpr(lr_5, 'temp')
plt.show()
sm.graphics.plot_ccpr(lr_5, 'Sep')
plt.show()
sm.graphics.plot_ccpr(lr_5, 'windspeed')
plt.show()

Linearity can be observed from above visualizations.

#### Homoscedasticity

In [ ]:
# Making polt for Residuals.
y_train_pred = lr_5.predict(X_train_lm_5)
residual = y_train - y_train_pred
sns.scatterplot(y_train,residual)
plt.plot(y_train,(y_train - y_train), '-r')
plt.xlabel('Count')
plt.ylabel('Residual')
plt.show()

No visible pattern observed from above plot for residuals.

#### Independence of residuals

Durbin-Watson value of final model lr_5 is 2.085, which signifies there is no autocorrelation.

# Step 10: Making Predictions Using the Final Model
Now that we have fitted the model and checked the normality of error terms, Now we will go ahead and make predictions using the final, i.e. 5th model.

In [ ]:
# Applying scaling on the Test Dataset.
num_vars = ['temp', 'humidity', 'windspeed','cnt']
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.head()

In [ ]:
# Checking the Description of the Columns of the Test Dataset.
df_test.describe()

In [ ]:
y_test = df_test.pop('cnt')
X_test = df_test

In [ ]:
col1 = X_train_new.columns
X_test = X_test[col1]
# Adding constant variable to Test Dataframe.
X_test_lm_5 = sm.add_constant(X_test)

In [ ]:
y_pred = lr_5.predict(X_test_lm_5)

In [ ]:
r2 = r2_score(y_test, y_pred)
round(r2,4)

# Step 11: Model Evaluation

Plot the graph for actual versus predicted values.

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test, y_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20) 
plt.xlabel('y_test', fontsize = 18)
plt.ylabel('y_pred', fontsize = 16) 

In [ ]:
round(lr_5.params,4)

We can see that the equation of our best fitted line is:

$ cnt = 0.1909 + 0.2341  \times  year - 0.0963  \times  holiday + 0.4777 \times temp - 0.1481 \times windspeed + 0.0910 \times sep - 0.2850 \times Light_snowrain - 0.0787 \times Misty - 0.0554 \times spring + 0.0621 \times summer + 0.0945 \times winter $

In [ ]:
# Calculating Adjusted-R^2 value for the test dataset
adjusted_r2 = round(1-(1-r2)*(X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1),4)
print(adjusted_r2)

In [ ]:
# Visualizing the fit on the Test Data.
# plotting a Regression plot
plt.figure()
sns.regplot(x=y_test, y=y_pred, ci=68, fit_reg=True,scatter_kws={"color": "blue"}, line_kws={"color": "red"})
plt.title('y_test vs y_pred', fontsize=20)
plt.xlabel('y_test', fontsize=18)
plt.ylabel('y_pred', fontsize=16)
plt.show()

# Comparision between Training and Testing dataset:
    - Train dataset R^2          : 0.833
    - Test dataset R^2           : 0.8038
    - Train dataset Adjusted R^2 : 0.829    
    - Test dataset Adjusted R^2  : 0.7944

#### Demand of bikes depend on year, holiday, temp, windspeed, sep, Light_snowrain, Misty, spring, summer and winter.

### Summary:

The summary of the model after data interpretation, visualisation, data-preparation, model building and training, residual analysis and evaluation of test model are as follows-

1) The R-squared value of the train set is 82.71% whereas the test set has a value of 81.13% which suggests that our model broadly explains the variance quite accurately on the test set and thus we can conclude that it is a good model. 

2) Our developed model's mean squared error is almost 0 on both the training and testing datasets which suggests that the variance is accurately predicted on the test set. The p-values and VIF were used to select the significant variables. RFE was also conducted for automated selection of variables.  

3) We can conclude that the bike demands for the BoomBikes is company is dependent on the temperature and whether it is a workingday or not. Additionally more rentals seem to be demanded on the winters as compared to the summer and spring. We had observed that the months of September and October had higher use of rentals. In terms of days the maximum focus was on days like Wed, Thurs and Sat and more on holidays. 

4) These interpretations help us derive meaningful insights in the bike rental market and the behaviour of the people. One of the recommendations based on this model are that there should be aggressive marketing in the summer and spring season to drive up rentals. Since the summer months also show low rental levels, a strong marketing strategy for the first 6 months of the year can assist in driving up the rental numbers. There has to be an approach required to introduce more users on days where the weather is less clear, perhaps with incentives or strategic deals. Rentals were more in 2019 than 2018 which suggests that over time more people would be exposed to this idea and there has to a strong analysis done to retain the repeat customers. 